@author: J. D. Camacho

# Notebook to search the best sub-model for a given dataset

This notebook will compute the best architecture and hyper-parameters for a model using the hyperband method

In [ ]:
# Import basic modules
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
%cd /app/

In [ ]:
from keras_autocompressor.hypermodels import HyperAutoCompressMobileNetV2

In [ ]:
# Experiment settings
tensorflow_dataset = 'horses_or_humans'
batch_size = 8

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    tensorflow_dataset, 
    split=['train', 'test'], 
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True
)

In [ ]:
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
def preprocess_images(image, label, classes=ds_info.features['label'].num_classes):
    # Resize images
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224,224))

    # Preprocess with the MobileNet function
    image = tf.keras.applications.mobilenet.preprocess_input(image)

    # Change labels to categorical
    label = tf.cast(tf.one_hot(tf.cast(label, tf.int32), classes), dtype=tf.float32)
    
    return image, label

In [ ]:
ds_train = ds_train.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Create pipeline for inference in both splits

ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)   
# ds_train = ds_train.cache()   
# ds_train = ds_train.prefetch(tf.data .AUTOTUNE)

ds_test = ds_test.batch(batch_size)
# ds_test = ds_test.cache()
# ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Hyper model
HyperModel = HyperAutoCompressMobileNetV2(
    max_parmeters=2309581,
    num_classes=ds_info.features['label'].num_classes,
    tau=0.4,
)

mytuner = kt.Hyperband(HyperModel,
                       max_epochs=5,
                       objective=kt.Objective("val_acc_comp", direction="max"),
                       directory='./logs/mobilenetv2/',
                       project_name=tensorflow_dataset,
                       overwrite=True)

In [ ]:
mytuner.search(ds_train, validation_data=ds_test)

In [ ]:
best_hp = mytuner.get_best_hyperparameters()[0]

In [ ]:
best_hp.values

In [ ]:
best_model = mytuner.get_best_models()[0]

In [ ]:
for sub_model in mytuner.get_best_models(15):
    metrics = sub_model.evaluate(val_ds, verbose=0)
    print('val_accuracy: {}  -- params: {}'.format(metrics[1], sub_model.count_params()))

In [ ]:
best_model.summary()

In [ ]:
best_model.evaluate(val_ds)

In [ ]:
mytuner.results_summary()